# Sync Validations - EXPORT

Export des opportunités depuis le Lakehouse vers Excel pour validation manuelle.

## Fréquence
- **CRON**: Hebdomadaire (lundi matin, après `deduplicate_weekly.ipynb`)

## Flux
```
landing_feedly_opportunities + landing_salers → Excel (nouveau fichier chaque semaine)
```

## Filtres appliqués
- `audit_opportunity = 1`
- `is_duplicate = false` (exclut les doublons confirmés)
- `ingestion_week = semaine courante`

## Fichier Excel
- **Emplacement**: `/lakehouse/default/Files/weak_signals_validation.xlsx`
- **Accès users**: OneLake File Explorer ou téléchargement depuis Fabric
- **Note**: Le fichier est SUPPRIMÉ et recréé chaque semaine (pas de cumul)

## 1. Configuration

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Semaine d'ingestion à exporter (format: "2024-W49")
# "auto" = semaine courante automatique (défaut)
INGESTION_WEEK = "auto"

# Chemin du fichier Excel dans le Lakehouse
EXCEL_PATH = "/lakehouse/default/Files/weak_signals_validation.xlsx"

# Tables Lakehouse
TABLE_OPPORTUNITIES = "landing_feedly_opportunities"
TABLE_SALERS = "landing_salers"
TABLE_SALERS_BD = "landing_salers_bd"

# Calculer la semaine courante si "auto"
from datetime import datetime
if INGESTION_WEEK == "auto":
    INGESTION_WEEK = datetime.now().strftime("%Y-W%V")

print(f"Semaine d'ingestion: {INGESTION_WEEK}")
print(f"Excel: {EXCEL_PATH}")

## 2. Imports et Setup

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, lower
from datetime import datetime
import pandas as pd
import os

spark = SparkSession.builder.getOrCreate()
print("Spark session ready")

## 3. Fonctions utilitaires

In [ ]:
def file_exists(path):
    """Vérifie si un fichier existe dans le Lakehouse."""
    try:
        mssparkutils.fs.head(path, 1)
        return True
    except:
        return os.path.exists(path)


def delete_file(path):
    """Supprime un fichier dans le Lakehouse."""
    try:
        mssparkutils.fs.rm(path, recurse=False)
        return True
    except:
        try:
            os.remove(path)
            return True
        except:
            return False


def get_opportunities_with_salers(ingestion_week):
    """
    Récupère les opportunités (audit_opportunity=1) avec les salers assignés.
    
    Filtres:
    - audit_opportunity = 1
    - is_duplicate = false OU NULL (exclut les doublons confirmés)
    - ingestion_week = semaine spécifiée
    
    Jointure sur country pour RAE/RSM.
    """
    query = f"""
    SELECT 
        o.id,
        o.ingestion_week,
        o.article_title,
        o.article_url,
        o.venue_name,
        o.city,
        o.country,
        o.vertical,
        o.evaluation_score,
        o.audit_opportunity_reason,
        -- Déduplication
        o.is_duplicate,
        o.is_suspected_duplicate,
        o.duplicate_score,
        -- Saler
        s.who AS saler_name,
        s.email AS saler_email
    FROM {TABLE_OPPORTUNITIES} o
    LEFT JOIN {TABLE_SALERS} s 
        ON TRIM(LOWER(o.country)) = TRIM(LOWER(s.country))
    WHERE o.audit_opportunity = 1
      AND (o.is_duplicate = false OR o.is_duplicate IS NULL)
      AND o.ingestion_week = '{ingestion_week}'
    ORDER BY o.evaluation_score DESC
    """
    return spark.sql(query)


print("Fonctions utilitaires chargées")

## 4. Export: Lakehouse → Excel

In [ ]:
def run_export():
    """
    Export les opportunités de la semaine vers Excel.
    
    IMPORTANT: Supprime l'Excel existant et crée un nouveau fichier
    uniquement avec les opportunités de la semaine courante.
    
    Filtres appliqués:
    - ingestion_week = INGESTION_WEEK
    - is_duplicate = false (exclut les doublons confirmés)
    - audit_opportunity = 1
    """
    print("=" * 60)
    print("EXPORT: Lakehouse → Excel")
    print("=" * 60)
    print(f"Semaine d'ingestion: {INGESTION_WEEK}")
    
    # 1. Supprimer l'Excel existant
    print("\n1. Suppression de l'Excel existant...")
    if file_exists(EXCEL_PATH):
        if delete_file(EXCEL_PATH):
            print(f"   ✓ Fichier supprimé: {EXCEL_PATH}")
        else:
            print(f"   ⚠️ Impossible de supprimer: {EXCEL_PATH}")
    else:
        print("   Aucun fichier existant.")
    
    # 2. Récupérer les opportunités avec salers (filtrées sur la semaine)
    print("\n2. Récupération des opportunités...")
    df_opportunities = get_opportunities_with_salers(ingestion_week=INGESTION_WEEK).toPandas()
    print(f"   {len(df_opportunities)} opportunités trouvées pour {INGESTION_WEEK}")
    
    if len(df_opportunities) == 0:
        print("   Aucune opportunité à exporter pour cette semaine.")
        return None
    
    # Stats déduplication
    n_suspected = len(df_opportunities[df_opportunities["is_suspected_duplicate"] == True])
    if n_suspected > 0:
        print(f"   ⚠️ {n_suspected} articles en zone grise (doublons potentiels)")
    
    # 3. Préparer le DataFrame avec colonnes validation vides
    print("\n3. Préparation des données...")
    df_export = df_opportunities.rename(columns={"id": "opportunity_id"})
    df_export["is_validated"] = None
    df_export["validation_comment"] = None
    df_export["validated_by"] = None
    df_export["email_sent_at"] = None
    
    # 4. Écrire le fichier Excel
    print("\n4. Écriture Excel...")
    
    # Ordre des colonnes
    columns_order = [
        "opportunity_id",
        "ingestion_week",
        "article_title",
        "article_url",
        "venue_name",
        "city",
        "country",
        "vertical",
        "evaluation_score",
        "audit_opportunity_reason",
        "is_duplicate",
        "is_suspected_duplicate",
        "duplicate_score",
        "saler_name",
        "saler_email",
        "is_validated",
        "validation_comment",
        "validated_by",
        "email_sent_at"
    ]
    
    # S'assurer que toutes les colonnes existent
    for c in columns_order:
        if c not in df_export.columns:
            df_export[c] = None
    
    df_export = df_export[columns_order]
    df_export.to_excel(EXCEL_PATH, index=False)
    
    print(f"   ✓ Fichier créé: {EXCEL_PATH}")
    print(f"   ✓ {len(df_export)} lignes exportées")
    
    return df_export


# Exécuter l'export
df_exported = run_export()

## 5. Statistiques Excel

In [ ]:
# Statistiques du fichier Excel
print("=" * 60)
print("STATISTIQUES EXCEL")
print("=" * 60)

if file_exists(EXCEL_PATH):
    df_stats = pd.read_excel(EXCEL_PATH)
    print(f"\nFichier: {EXCEL_PATH}")
    print(f"  Total lignes: {len(df_stats)}")
    print(f"  Validées (OK): {len(df_stats[df_stats['is_validated'] == 1])}")
    print(f"  Rejetées (KO): {len(df_stats[df_stats['is_validated'] == 0])}")
    print(f"  En attente: {len(df_stats[df_stats['is_validated'].isna()])}")
    
    # Stats par semaine
    if "ingestion_week" in df_stats.columns:
        print(f"\n  Par semaine:")
        week_stats = df_stats.groupby("ingestion_week").agg({
            "opportunity_id": "count",
            "is_validated": lambda x: x.notna().sum()
        }).rename(columns={"opportunity_id": "total", "is_validated": "validées"})
        print(week_stats.to_string())
    
    # Stats zone grise
    if "is_suspected_duplicate" in df_stats.columns:
        n_grey = len(df_stats[df_stats["is_suspected_duplicate"] == True])
        if n_grey > 0:
            print(f"\n   {n_grey} articles en zone grise")
else:
    print(f"Fichier non trouvé: {EXCEL_PATH}")

## 6. Aperçu des données exportées

In [ ]:
# Aperçu des premières lignes
if df_exported is not None:
    print("Aperçu des données exportées:")
    display(df_exported[["opportunity_id", "ingestion_week", "venue_name", "city", "country", "evaluation_score", "is_validated"]].head(20))

---

## Guide d'utilisation

### Configuration

| Variable | Description | Exemple |
|----------|-------------|--------|
| `INGESTION_WEEK` | Semaine à exporter | `"2024-W49"` ou `"auto"` (défaut) |

### Comportement

- **L'Excel existant est SUPPRIMÉ** à chaque exécution
- Un nouveau fichier est créé avec uniquement la semaine spécifiée
- Pas de cumul des semaines précédentes

### Après l'export

1. Télécharger le fichier Excel depuis Fabric ou OneLake File Explorer
2. Valider les opportunités:
   - `is_validated`: 1 (OK) ou 0 (KO)
   - `validation_comment`: commentaire optionnel
   - `validated_by`: votre email
3. Uploader le fichier Excel modifié
4. Exécuter `sync_validations_import.ipynb`

### Colonnes Excel

| Colonne | Description | Rempli par |
|---------|-------------|------------|
| `opportunity_id` | ID unique | Auto |
| `ingestion_week` | Semaine d'ingestion | Auto |
| `article_title` | Titre de l'article | Auto |
| `venue_name` | Nom du lieu | Auto |
| `is_suspected_duplicate` | Zone grise | Auto (True/False) |
| `is_validated` | **Validation** | **User** (1=OK, 0=KO) |
| `validation_comment` | Commentaire | User |
| `validated_by` | Email validateur | User |